# Named Entity Recognition 

In [1]:
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd
import logging
import sys

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
data_dir = "/dataset/nergrit-corpus/ner/data"
file_train_corrected = f'{data_dir}/train_corrected.txt'
file_valid_corrected = f'{data_dir}/valid_corrected.txt'
file_test_corrected = f'{data_dir}/test_corrected.txt'
file_labels_map = f'{data_dir}/labels-map.csv'

In [4]:
train_data = []
sentence_counter = 0
with open(file_train_corrected) as fp:
    for cnt, line in enumerate(fp):
        try:
            texts = line.split()
            sentence_id, words, labels = texts[0], ' '.join(texts[1:-1]), texts[-1]
            train_data.append([int(sentence_id), words, labels])
        except:
            pass

train_data = pd.DataFrame(
    train_data, columns=["sentence_id", "words", "labels"],
)

In [5]:
valid_data = []
sentence_counter = 0
with open(file_valid_corrected) as fp:
    for cnt, line in enumerate(fp):
        try:
            texts = line.split()
            sentence_id, words, labels = texts[0], ' '.join(texts[1:-1]), texts[-1]
            valid_data.append([int(sentence_id), words, labels])
        except:
            pass

valid_data = pd.DataFrame(
    valid_data, columns=["sentence_id", "words", "labels"],
)

In [6]:
test_data = []
sentence_counter = 0
with open(file_test_corrected) as fp:
    for cnt, line in enumerate(fp):
        try:
            texts = line.split()
            sentence_id, words, labels = texts[0], ' '.join(texts[1:-1]), texts[-1]
            test_data.append([int(sentence_id), words, labels])
        except:
            pass

test_data = pd.DataFrame(
    test_data, columns=["sentence_id", "words", "labels"],
)

In [7]:
len(train_data),len(valid_data),len(test_data)

(309203, 61680, 64568)

In [8]:
train_data.head(10)

,sentence_id,words,labels
0,0,Indonesia,B-GPE
1,0,mengekspor,O
2,0,produk,O
3,0,industri,O
4,0,skala,O
5,0,besar,O
6,0,ke,O
7,0,Amerika,B-GPE
8,0,Serikat,I-GPE
9,0,.,O


In [9]:
valid_data.head(10)

,sentence_id,words,labels
0,0,Rupiah,O
1,0,Terus,O
2,0,Melemah,O
3,0,",",O
4,0,BI,B-ORG
5,0,:,O
6,0,Kebijakan,O
7,0,Moneter,O
8,0,Sudah,O
9,0,Tepat,O


In [10]:
df = pd.read_csv(file_labels_map, sep=' ', names=['X','Y'])
labels = list(set(df['Y']))
labels.sort()
len(labels)

39

In [11]:
labels

['B-CRD',
 'B-DAT',
 'B-EVE',
 'B-FAC',
 'B-GPE',
 'B-LAN',
 'B-LAW',
 'B-LOC',
 'B-MON',
 'B-NOR',
 'B-ORD',
 'B-ORG',
 'B-PER',
 'B-PRC',
 'B-PRD',
 'B-QTY',
 'B-REG',
 'B-TIM',
 'B-WOA',
 'I-CRD',
 'I-DAT',
 'I-EVE',
 'I-FAC',
 'I-GPE',
 'I-LAN',
 'I-LAW',
 'I-LOC',
 'I-MON',
 'I-NOR',
 'I-ORD',
 'I-ORG',
 'I-PER',
 'I-PRC',
 'I-PRD',
 'I-QTY',
 'I-REG',
 'I-TIM',
 'I-WOA',
 'O']

In [12]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/tmp/outputs/'
model_args.best_model_dir = '/tmp/outputs/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels

In [13]:
model_args

NERArgs(adam_epsilon=1e-08, best_model_dir='/tmp/outputs/best_model', cache_dir='cache_dir/', config={}, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=78, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=True, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, fp16=False, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=128, model_name=None, model_type=None, multiprocessing_chunksize=500, n_gpu=1, no_cache=False, no_save=False, num_train_epochs=5, output_dir='/tmp/outputs/', overwrite_output_dir=True, process_count=78, quantized_model=False, reprocess_input_data=True, save_best_model=True, sa

In [14]:
model = NERModel(
    "bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [15]:
# Train the model
model.train_model(train_data, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /tmp/outputs/.


In [16]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.39721426626775735, 'precision': 0.5692018571744417, 'recall': 0.6105774931815486, 'f1_score': 0.5891641398249328}


In [17]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.38367636976692515, 'precision': 0.5665203895964893, 'recall': 0.5953880764904387, 'f1_score': 0.5805956233203532}


In [18]:
# Make predictions with the model
texts = [
    "Gubernur Bank Indonesia Agus Martowardojo bersama jajaran deputi Gubernur Bank Indonesia menggelar konferensi pers usai Rapat Dewan Gubernur di Bank Indonesia, Jakarta, Kamis (17/5/2015)",
    "Selama 24 jam puncak Mahameru di Malang kebanjiran pendaki dari Wina",
]
predictions, raw_outputs = model.predict(texts)

INFO:simpletransformers.ner.ner_model: Converting to features started.


In [19]:
predictions

[[{'Gubernur': 'B-NOR'},
  {'Bank': 'I-NOR'},
  {'Indonesia': 'I-NOR'},
  {'Agus': 'I-PER'},
  {'Martowardojo': 'I-PER'},
  {'bersama': 'O'},
  {'jajaran': 'O'},
  {'deputi': 'B-NOR'},
  {'Gubernur': 'I-NOR'},
  {'Bank': 'I-NOR'},
  {'Indonesia': 'I-NOR'},
  {'menggelar': 'O'},
  {'konferensi': 'B-EVE'},
  {'pers': 'I-EVE'},
  {'usai': 'O'},
  {'Rapat': 'B-EVE'},
  {'Dewan': 'I-EVE'},
  {'Gubernur': 'I-EVE'},
  {'di': 'O'},
  {'Bank': 'B-LOC'},
  {'Indonesia,': 'I-LOC'},
  {'Jakarta,': 'B-GPE'},
  {'Kamis': 'B-DAT'},
  {'(17/5/2015)': 'I-DAT'}],
 [{'Selama': 'O'},
  {'24': 'B-QTY'},
  {'jam': 'I-QTY'},
  {'puncak': 'O'},
  {'Mahameru': 'I-LOC'},
  {'di': 'O'},
  {'Malang': 'B-LOC'},
  {'kebanjiran': 'O'},
  {'pendaki': 'O'},
  {'dari': 'O'},
  {'Wina': 'B-LOC'}]]